# CPE KMUTT website scraper

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import re
import pandas as pd
#from selenium.webdriver.edge.options import Options

In [2]:
driver = webdriver.Edge(executable_path = 'edgedriver_win64\\msedgedriver.exe')

<h2> Name, Position, and URLs </h2>

In [3]:
url = 'https://www.cpe.kmutt.ac.th/th/staff'
driver.get(url)
sleep(0.5)
# get HTML from driver at current page
soup = BeautifulSoup(driver.page_source,'html.parser')
# declare lists
position = []
postid = []
fullname = []
for i in range(0, len(soup.select('p.text-muted'))):
    position.append(soup.select('p.text-muted')[i].get_text())
    fullname.append(soup.select('h3.profile-username')[i].get_text())
    tmp = re.search(r'(postid)=\"\S+\"', str(soup.select('h3.profile-username')[i]))
    postid.append(tmp.group(0)[8:-1])

personnel = pd.DataFrame({'PostID': postid,
                          'Position': position,
                          'FullName': fullname})

## Education and Research field

In [4]:
# Use .format to fill url for the best time-complexity
urls = url + "-detail/{}"
education = list()
role, tel, email = [],[],[] 
for idx in postid:
    driver.get(urls.format(idx))
    sleep(2)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    info = soup.select('p.staff-list-detail')
    edu = soup.select('div.col-xs-12.post-content')[0].get_text()
    education.append(edu)
    role.append(info[0].get_text())
    tel.append(info[1].get_text())
    email.append(info[2].get_text())

moreinfo = pd.DataFrame({'role':role, 
                         'tel':tel,
                         'email':email,
                         'education':education})

## Constructing DataFrame

In [68]:
df = pd.concat([personnel, moreinfo], axis = 1)
df

,PostID,Position,FullName,role,tel,email,education
0,Sanan,ผู้ช่วยศาสตราจารย์,ผศ.สนั่น สระแก้ว,หัวหน้าภาควิชา,+662 470 9254,sanan.sra@mail.kmutt.ac.th,"Education\n\nM.S. (George Washington U., USA)\..."
1,Pipat,ผู้ช่วยศาสตราจารย์,ผศ.พิพัฒน์ ศุภศิริสันต์,เลขานุการภาควิชา,+662 470 9082,pipat@cpe.kmutt.ac.th,Education\n\nB.Eng. (KMITT)\n\nResearch Intere...
2,Booncharoen,ศาสตราจารย์,ศ.ดร.บุญเจริญ ศิริเนาวกุล,อาจารย์,+662 470 9255,boon@kmutt.ac.th,Education\n\nD.Eng. (KMITL)\nM.S. (The Wichita...
3,Tiranee,รองศาสตราจารย์,รศ.ดร.ธีรณี อจลากุล,อาจารย์,+662 470 9380,tiranee.ach@mail.kmutt.ac.th,"Education\n\nPh.D. in Computer Engineering, Sy..."
4,Peerapon,รองศาสตราจารย์,รศ.ดร.พีรพล ศิริพงศ์วุฒิกร,อาจารย์,+662 470 9381,peerapon.sir@mail.kmutt.ac.th,"Education\n\nPh.D. in Telecommunications, 2003..."
5,Thumrongrat,รองศาสตราจารย์,รศ.ดร.ธำรงรัตน์ อมรรักษา,อาจารย์,+662 470 9083,t_amornraksa@cpe.kmutt.ac.th,Education\n\nPh.D. degree in Electronic and El...
6,Natasha,รองศาสตราจารย์,รศ.ดร.ณัฐชา เดชดำรง,อาจารย์,+662 470 9390,natasha@cpe.kmutt.ac.th,"Education\n\nB.Sc. Computer Science, Thammasat..."
7,Naruemon,รองศาสตราจารย์,รศ.ดร.นฤมล วัฒนพงศกร,อาจารย์,+662 470 9089,naruemon@cpe.kmutt.ac.th,"Education\n\nPh.D. in Electrical Engineering, ..."
8,Bundit,รองศาสตราจารย์﻿,รศ.ดร.บัณฑิต ทิพากร,อาจารย์,+662 470 9081,bundit@cpe.kmutt.ac.th,"Education\n\nPh.D. (Missouri-Columbia, USA)\nM..."
9,Nuttanart,ผู้ช่วยศาสตราจารย์,ผศ.ดร.ณัฐนาถ ฟาคุนเด๊ซ,อาจารย์,+662 470 9256,nuttanart@cpe.kmutt.ac.th,"Education\n\nPh.D. (Linguistics, The state U. ..."


In [62]:
test = df.iloc[9,3].split('\n')
list(filter(lambda x: x != '', test))

['Education',
 'Ph.D. (Linguistics, The state U. of NY)',
 'M.A. (Linguistics, The state U. of NY)',
 'B.A. (CU)',
 'Research Interests',
 'Natural Language Processing',
 'Linguistic-based computation',
 'Information Retrieval']